In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\drewa\.wdm\drivers\chromedriver\win32\101.0.4951.41]


## Sex Mortality Data

In [3]:
# Visit the State Cancer Profile(Incidence) site
url = 'https://www.statecancerprofiles.cancer.gov/deathrates/index.php'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.col-xs-9', wait_time=1)

True

In [4]:
# Set the html parameter of interest
html = browser.html
can_soup = soup(html, 'html.parser')
slide_elem = can_soup.select_one('fieldset')

In [5]:
print(slide_elem.prettify())

<fieldset>
 <legend>
  Data Options
 </legend>
 <div class="inputWrap cf">
  <label for="stateFIPS">
   <strong>
    Area:
   </strong>
  </label>
  <select id="stateFIPS" name="stateFIPS" onchange="updateAreaType();">
   <option selected="selected" value="*">
    --- choose state ---
   </option>
   <option value="00">
    United States
   </option>
   <option value="01">
    Alabama
   </option>
   <option value="02">
    Alaska
   </option>
   <option value="04">
    Arizona
   </option>
   <option value="05">
    Arkansas
   </option>
   <option value="06">
    California
   </option>
   <option value="08">
    Colorado
   </option>
   <option value="09">
    Connecticut
   </option>
   <option value="10">
    Delaware
   </option>
   <option value="11">
    District of Columbia
   </option>
   <option value="12">
    Florida
   </option>
   <option value="13">
    Georgia
   </option>
   <option value="15">
    Hawaii
   </option>
   <option value="16">
    Idaho
   </option>
   <

In [6]:
slide_elem.find_all('option')

[<option selected="selected" value="*">--- choose state ---</option>,
 <option value="00">United States</option>,
 <option value="01">Alabama</option>,
 <option value="02">Alaska </option>,
 <option value="04">Arizona</option>,
 <option value="05">Arkansas</option>,
 <option value="06">California</option>,
 <option value="08">Colorado</option>,
 <option value="09">Connecticut</option>,
 <option value="10">Delaware</option>,
 <option value="11">District of Columbia</option>,
 <option value="12">Florida</option>,
 <option value="13">Georgia</option>,
 <option value="15">Hawaii </option>,
 <option value="16">Idaho</option>,
 <option value="17">Illinois</option>,
 <option value="18">Indiana</option>,
 <option value="19">Iowa </option>,
 <option value="20">Kansas </option>,
 <option value="21">Kentucky</option>,
 <option value="22">Louisiana </option>,
 <option value="23">Maine</option>,
 <option value="24">Maryland</option>,
 <option value="25">Massachusetts</option>,
 <option value="26">M

In [7]:
# Cancer Incidence Data(2015-2019)

# Data options lists
#Area
state = {}
results = slide_elem.find_all('option')[2:54]
for result in results:
    state[result.attrs['value']] = result.text
state_keys = list(state)
# Cancer
cancer = {}
results = slide_elem.find_all('option')[59:80]
for result in results:
    cancer[result.attrs['value']] = result.text
cancer_keys = list(cancer)
# Race/Ethnicity
race = {}
results = slide_elem.find_all('option')[82:89]
for result in results:
    race[result.attrs['value']] = result.text
race_keys = list(race)
# Sex
sex = {}
results = slide_elem.find_all('option')[91:93]
for result in results:
    sex[result.attrs['value']] = result.text
sex_keys = list(sex)
# Age
age = {}
results = slide_elem.find_all('option')[96:99]
for result in results:
    age[result.attrs['value']] = result.text
age_keys = list(age)

In [8]:
sex_keys

['1', '2']

In [9]:
# Reset dictionaries
cancer_reset = {'001':'All Cancer Sites'}
race_reset = {'00':'All Races (includes Hispanic)'}
sex_reset = {'0':'Both Sexes'}
age_reset = {'001':'All Ages'}

In [10]:
# site = f'https://www.statecancerprofiles.cancer.gov/incidencerates/index.php?stateFIPS={state_num}&areatype=county&cancer={cancer_num}&race={race_num}&sex={sex_num}&age={age_num}&stage=999&year=0&type=incd&sortVariableName=rate&sortOrder=default&output=0#results'

# For loop to change all data options to relevant data points

# Create empty dataframe
df = []

# Set initial parameters
cancer_num = list(cancer_reset.keys())[0]
race_num = list(race_reset.keys())[0]
sex_num = list(sex_reset.keys())[0]
age_num = list(age_reset.keys())[0]
print(cancer_num, race_num, sex_num, age_num)

001 00 0 001


In [11]:
sex_mort_list = []
# Select Sex features
cancer_num = list(cancer_reset.keys())[0]
race_num = list(race_reset.keys())[0]
sex_num = list(sex_reset.keys())[0]
age_num = list(age_reset.keys())[0]
for a in range(len(state_keys)):
    state_num = state_keys[a]
    for d in range(len(sex_keys)):
        # URL variable
        sex_num = sex_keys[d]
        # Visit site
        url_to_visit = f'https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS={state_num}'
        url_to_visit += f'&areatype=county&cancer={cancer_num}'
        url_to_visit += f'&race={race_num}'
        url_to_visit += f'&sex={sex_num}'
        url_to_visit += f'&age={age_num}'
        url_to_visit += f'&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results'
        print("cancer_num", url_to_visit)
        
        browser.visit(url_to_visit)
        # Wait for load
        browser.is_element_present_by_css('div.col-xs-9', wait_time=4)
        url2= f'https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS={state_num}'
        url2 += f'&areatype=county&cancer={cancer_num}'
        url2 += f'&race={race_num}'
        url2 += f'&sex={sex_num}'
        url2 += f'&age={age_num}'
        url2 += '&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results'
        
        print("url2", url2)
        y = pd.read_html(url2)[0]
        # Add selection criteria to y
        y.reindex(columns = y.columns.tolist() + ["State", "Cancer", "Race/Ethnicity", "Sex", "Age"])
        y['State'] = state[state_keys[a]]
        y['Cancer'] = "All Cancer Sites"
        y['Race/Ethnicity'] = "All Races (includes Hispanic)"
        y['Sex'] = sex[sex_keys[d]]
        y['Age'] = 'All Ages'
        # Add to current dataframe
        display(y)
        sex_mort_list.append(y)
        # Reset conditionals
        cancer_num = list(cancer_reset.keys())[0]
        sex_num = list(sex_reset.keys())[0]
        age_num = list(age_reset.keys())[0]
browser.quit()

cancer_num https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=01&areatype=county&cancer=001&race=00&sex=1&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results
url2 https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=01&areatype=county&cancer=001&race=00&sex=1&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results


,County,Met Healthy People Objective of ***?,"Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)",CI*Rank&fork;(95% Confidence Interval),Average Annual Count,Recent Trend,Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval),State,Cancer,Race/Ethnicity,Sex,Age
0,Alabama,***,"212.1 (209.6, 214.7)",NaN,5695,falling,"-2.2 (-2.5, -1.9)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
1,United States,***,"181.4 (181.1, 181.7)",NaN,314987,falling,"-2.3 (-2.6, -2.0)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
2,Wilcox County,***,"296.3 (235.0, 369.2)","1 (1, 55)",18,stable,"-0.3 (-1.5, 0.9)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
3,Macon County,***,"290.7 (245.1, 343.0)","2 (1, 33)",32,falling,"-1.0 (-1.7, -0.3)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
4,Lowndes County,***,"283.8 (224.3, 355.1)","3 (1, 60)",17,stable,"-0.7 (-1.7, 0.4)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
...,...,...,...,...,...,...,...,...,...,...,...,...
64,Limestone County,***,"190.0 (172.5, 208.8)","63 (35, 66)",95,falling,"-2.9 (-4.1, -1.7)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
65,Cleburne County,***,"188.0 (150.7, 232.6)","64 (9, 67)",19,falling,"-1.7 (-2.9, -0.5)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
66,Madison County,***,"180.9 (172.1, 190.0)","65 (54, 66)",337,falling,"-2.2 (-2.8, -1.5)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
67,Greene County,***,"176.6 (130.3, 236.1)","66 (8, 67)",10,falling,"-2.3 (-3.6, -1.0)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages


cancer_num https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=01&areatype=county&cancer=001&race=00&sex=2&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results
url2 https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=01&areatype=county&cancer=001&race=00&sex=2&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results


,County,Met Healthy People Objective of ***?,"Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)",CI*Rank&fork;(95% Confidence Interval),Average Annual Count,Recent Trend,Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval),State,Cancer,Race/Ethnicity,Sex,Age
0,Alabama,***,"139.2 (137.4, 141.0)",NaN,4722,falling,"-1.3 (-1.5, -1.1)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
1,United States,***,"131.1 (130.9, 131.3)",NaN,283394,falling,"-1.9 (-2.1, -1.6)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
2,Bullock County,***,"191.5 (146.3, 247.7)","1 (1, 60)",13,stable,"0.7 (-0.6, 2.0)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
3,Crenshaw County,***,"179.5 (143.3, 223.1)","2 (1, 59)",18,stable,"0.4 (-0.6, 1.3)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
4,Fayette County,***,"176.0 (144.2, 213.8)","3 (1, 56)",23,stable,"0.6 (-0.5, 1.6)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
...,...,...,...,...,...,...,...,...,...,...,...,...
64,Limestone County,***,"113.3 (101.6, 126.1)","63 (48, 67)",70,falling,"-2.3 (-3.8, -0.9)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
65,Pickens County,***,"111.8 (89.3, 139.2)","64 (21, 67)",18,falling,"-1.1 (-2.0, -0.2)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
66,Choctaw County,***,"110.8 (83.6, 145.6)","65 (11, 67)",12,falling,"-1.2 (-2.1, -0.3)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
67,Winston County,***,"107.4 (87.3, 131.5)","66 (34, 67)",21,falling,"-1.4 (-2.5, -0.3)",Alabama,All Cancer Sites,All Races (includes Hispanic),Females,All Ages


cancer_num https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=02&areatype=county&cancer=001&race=00&sex=1&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results
url2 https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=02&areatype=county&cancer=001&race=00&sex=1&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results


...

,County,Met Healthy People Objective of ***?,"Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)",CI*Rank&fork;(95% Confidence Interval),Average Annual Count,Recent Trend,Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval),State,Cancer,Race/Ethnicity,Sex,Age
0,Wisconsin,***,"133.0 (131.4, 134.6)",NaN,5358,falling,"-1.7 (-2.1, -1.2)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
1,United States,***,"131.1 (130.9, 131.3)",NaN,283394,falling,"-1.9 (-2.1, -1.6)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
2,Florence County,***,"199.6 (130.1, 300.5)","1 (1, 71)",7,stable,"0.9 (-0.8, 2.7)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
3,Washburn County,***,"172.8 (140.8, 211.4)","2 (1, 53)",25,stable,"-0.3 (-1.3, 0.8)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
4,Burnett County,***,"172.2 (140.8, 210.4)","3 (1, 56)",24,stable,"0.1 (-0.9, 1.1)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
...,...,...,...,...,...,...,...,...,...,...,...,...
69,Iowa County,***,"113.0 (91.0, 139.4)","68 (12, 72)",19,falling,"-1.5 (-2.3, -0.6)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
70,Bayfield County,***,"108.6 (85.8, 138.1)","69 (16, 72)",16,falling,"-2.6 (-3.8, -1.4)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
71,Washington County,***,"108.0 (98.9, 117.8)","70 (51, 72)",108,falling,"-3.6 (-5.5, -1.7)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
72,Jackson County,***,"104.2 (81.8, 131.7)","71 (17, 72)",16,falling,"-1.5 (-2.5, -0.5)",Wisconsin,All Cancer Sites,All Races (includes Hispanic),Females,All Ages


cancer_num https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=56&areatype=county&cancer=001&race=00&sex=1&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results
url2 https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=56&areatype=county&cancer=001&race=00&sex=1&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results


,County,Met Healthy People Objective of ***?,"Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)",CI*Rank&fork;(95% Confidence Interval),Average Annual Count,Recent Trend,Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval),State,Cancer,Race/Ethnicity,Sex,Age
0,Wyoming,***,"160.2 (153.9, 166.7)",NaN,527,falling,"-1.8 (-2.0, -1.6)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
1,United States,***,"181.4 (181.1, 181.7)",NaN,314987,falling,"-2.3 (-2.6, -2.0)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
2,Big Horn County,***,"195.4 (153.3, 246.4)","1 (1, 17)",16,falling,"-1.2 (-2.3, -0.2)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
3,Goshen County,***,"195.1 (157.7, 239.8)","2 (1, 16)",20,falling,"-1.3 (-2.3, -0.3)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
4,Sweetwater County,***,"194.1 (164.7, 227.0)","3 (1, 14)",37,stable,"-0.8 (-1.7, 0.1)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
5,Platte County,***,"186.6 (144.5, 240.0)","4 (1, 18)",14,falling,"-2.1 (-3.4, -0.9)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
6,Crook County,***,"185.7 (136.7, 248.8)","5 (1, 19)",10,*,*,Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
7,Sheridan County,***,"184.9 (158.9, 214.3)","6 (1, 15)",38,falling,"-1.7 (-2.6, -0.9)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
8,Campbell County,***,"184.3 (152.4, 220.2)","7 (1, 16)",31,stable,"-1.4 (-2.7, 0.0)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages
9,Fremont County,***,"180.5 (156.7, 207.1)","8 (1, 15)",44,falling,"-1.0 (-1.9, -0.1)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Males,All Ages


cancer_num https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=56&areatype=county&cancer=001&race=00&sex=2&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results
url2 https://www.statecancerprofiles.cancer.gov/deathrates/index.php?stateFIPS=56&areatype=county&cancer=001&race=00&sex=2&age=001&year=0&type=death&sortVariableName=rate&sortOrder=default&output=0#results


,County,Met Healthy People Objective of ***?,"Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)",CI*Rank&fork;(95% Confidence Interval),Average Annual Count,Recent Trend,Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval),State,Cancer,Race/Ethnicity,Sex,Age
0,Wyoming,***,"121.7 (116.6, 127.1)",NaN,444,falling,"-1.6 (-1.9, -1.4)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
1,United States,***,"131.1 (130.9, 131.3)",NaN,283394,falling,"-1.9 (-2.1, -1.6)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
2,Hot Springs County,***,"209.5 (152.5, 288.9)","1 (1, 8)",10,stable,"0.6 (-0.9, 2.2)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
3,Campbell County,***,"163.3 (136.5, 193.5)","2 (1, 11)",29,stable,"-0.5 (-1.8, 0.8)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
4,Weston County,***,"149.4 (103.1, 211.9)","3 (1, 20)",8,stable,"-1.0 (-2.7, 0.7)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
5,Uinta County,***,"143.8 (112.8, 180.7)","4 (1, 17)",16,falling,"-1.4 (-2.6, -0.3)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
6,Johnson County,***,"140.7 (102.6, 191.3)","5 (1, 19)",10,stable,"-1.3 (-3.1, 0.6)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
7,Goshen County,***,"137.3 (106.9, 175.5)","6 (1, 19)",15,stable,"0.7 (-0.5, 1.9)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
8,Sweetwater County,***,"135.5 (113.4, 160.7)","7 (2, 17)",29,stable,"-0.7 (-1.6, 0.2)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages
9,Fremont County,***,"133.0 (113.6, 155.0)","8 (2, 16)",36,falling,"-1.3 (-2.1, -0.5)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages


In [14]:
# Make DataFrame
sex_mort_df = pd.concat(sex_mort_list)
sex_mort_df

,County,Met Healthy People Objective of ***?,"Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)",CI*Rank&fork;(95% Confidence Interval),Average Annual Count,Recent Trend,Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval),State,Cancer,Race/Ethnicity,Sex,Age,Borough Or Census Area,Parish
0,Alabama,***,"212.1 (209.6, 214.7)",NaN,5695,falling,"-2.2 (-2.5, -1.9)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,NaN,NaN
1,United States,***,"181.4 (181.1, 181.7)",NaN,314987,falling,"-2.3 (-2.6, -2.0)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,NaN,NaN
2,Wilcox County,***,"296.3 (235.0, 369.2)","1 (1, 55)",18,stable,"-0.3 (-1.5, 0.9)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,NaN,NaN
3,Macon County,***,"290.7 (245.1, 343.0)","2 (1, 33)",32,falling,"-1.0 (-1.7, -0.3)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,NaN,NaN
4,Lowndes County,***,"283.8 (224.3, 355.1)","3 (1, 60)",17,stable,"-0.7 (-1.7, 0.4)",Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20,Lincoln County,***,"88.8 (65.9, 117.6)","19 (10, 22)",11,falling,"-2.4 (-3.7, -1.2)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,NaN,NaN
21,Albany County,***,"86.1 (66.6, 109.5)","20 (13, 22)",14,falling,"-2.4 (-3.2, -1.6)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,NaN,NaN
22,Sublette County,***,"83.5 (53.7, 125.6)","21 (7, 22)",5,falling,"-3.4 (-5.4, -1.4)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,NaN,NaN
23,Teton County,***,"78.0 (57.1, 104.3)","22 (15, 22)",10,stable,"21.8 (-14.4, 73.4)",Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,NaN,NaN


In [15]:
sex_mort_df.to_csv('sex_mort_unclean.csv')

In [17]:
sex_mort_df1 =sex_mort_df.drop("Met Healthy People Objective of ***?", axis='columns')
sex_mort_df2 = sex_mort_df1.drop("Borough Or Census Area", axis='columns')
sex_mort_df3 = sex_mort_df2.drop("Parish", axis='columns')
sex_mort_df4 = sex_mort_df3.dropna()
sex_mort_df5 = sex_mort_df4.drop("CI*Rank&fork;(95% Confidence Interval)", axis='columns')
sex_mort_df5['Age-Adjusted Death Rate cases per 100,000'] = sex_mort_df5['Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)'].str.replace(r'\(.*\)', '')
sex_mort_df5['Recent 5-Year Trend in Death Rates'] = sex_mort_df5['Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)'].str.replace(r'\(.*\)', '')
sex_mort_df6 = sex_mort_df5.drop(["Age-Adjusted Death Rate†deaths per 100,000(95% Confidence Interval)", 'Recent 5-Year Trend‡ in Death Rates(95% Confidence Interval)'], axis='columns')
sex_mort_df7 = sex_mort_df6[sex_mort_df6['Average Annual Count'] != '3 or fewer']
sex_mort_df8 = sex_mort_df7.drop(['Recent Trend','Recent 5-Year Trend in Death Rates'], axis='columns')
sex_mort_df8

C:\Users\drewa\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
C:\Users\drewa\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,County,Average Annual Count,State,Cancer,Race/Ethnicity,Sex,Age,"Age-Adjusted Death Rate cases per 100,000"
2,Wilcox County,18,Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,296.3
3,Macon County,32,Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,290.7
4,Lowndes County,17,Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,283.8
5,Walker County,111,Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,281.1
6,Dallas County,60,Alabama,All Cancer Sites,All Races (includes Hispanic),Males,All Ages,271.3
...,...,...,...,...,...,...,...,...
19,Platte County,8,Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,101.1
20,Lincoln County,11,Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,88.8
21,Albany County,14,Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,86.1
22,Sublette County,5,Wyoming,All Cancer Sites,All Races (includes Hispanic),Females,All Ages,83.5


In [18]:
sex_mort_df8.to_csv('sex_mort.csv', index=False)